In [1]:
import arcpy
from arcpy import env
import pandas as pd
import numpy as np

In [2]:
env_path = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb"
env.workspace = env_path
raster_list = arcpy.ListRasters()
print(raster_list)
print(len(raster_list))

['DEM', 'AnalyticalHillshading', 'Aspect', 'ChannelNetworkBaseLevel', 'ChannelNetworkDistance', 'ClosedDepressions', 'ConvergenceIndex', 'LSFactor', 'MRRTF', 'MRVBF', 'PlanCurvature', 'ProfileCurvature', 'RelativeSlopePosition', 'Slope', 'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth', 'NIGHT2022', 'ETP2022_mean', 'TMP2022_mean', 'PRE2022_mean', 'PRE2022_3', 'PRE2022_11', 'ETP2022_3', 'ETP2022_11', 'TMP2022_3', 'TMP2022_11', 'evi', 'lswi', 'mndwi', 'ndmi', 'ndvi', 'ndwi', 'PCA_0', 'PCA_1', 'savi', 'vari', 'DL', 'DZ']
39


In [3]:
# 检查现有的raster空间范围和像元是否一致

def check_raster_info(raster_path):
    desc = arcpy.Describe(raster_path)
    origin_x = desc.extent.XMin
    origin_y = desc.extent.YMax
    pixel_width = desc.meanCellWidth
    pixel_height = desc.meanCellHeight
    rows = desc.height
    cols = desc.width
    print(origin_x,origin_y,rows,cols,pixel_width,pixel_height)
    return((origin_x,origin_y,rows,cols,pixel_width,pixel_height))

In [4]:
check_list = set()
for one_raster in raster_list:
    check_raster_info(one_raster)
    check_list.add(check_raster_info(one_raster))

411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 13343 5.0 5.0
411337.5 2924552.5 12881 

In [5]:
check_list

{(411337.5, 2924552.5, 12881, 13343, 5.0, 5.0)}

In [6]:
# 选取标准数据生成lon、lat图层
stander_raster = "DEM"
stander_desc = arcpy.Describe(stander_raster)
stander_origin_x = stander_desc.extent.XMin
stander_origin_y = stander_desc.extent.YMax
stander_pixel_width = stander_desc.meanCellWidth
stander_pixel_height = stander_desc.meanCellHeight
stander_array = arcpy.RasterToNumPyArray("DEM")

In [7]:
# 经度
array_x = np.zeros(stander_array.shape,dtype=np.float32)
array_x[:, 0] = stander_origin_x+stander_pixel_width/2
for i in range(1, stander_array.shape[1]):
    array_x[:, i] = array_x[:, i-1] + stander_pixel_width
print(array_x.shape,array_x[0,0],array_x[-1,-1])

(12881, 13343) 411340.0 478050.0


In [8]:
# 纬度
array_y = np.zeros(stander_array.shape,dtype=np.float32)
array_y[0] = stander_origin_y-stander_pixel_height/2
for i in range(1, stander_array.shape[0]):
    array_y[i] = array_y[i-1] - stander_pixel_height
print(array_y.shape,array_y[0][0],array_y[-1][-1])

(12881, 13343) 2924550.0 2860150.0


In [9]:
# 确定范围
env.extent = "DEM"

In [11]:
original_raster = arcpy.Raster("DEM")

In [12]:
# 将生成的x,y数组保存为栅格
out_raster_x = arcpy.NumPyArrayToRaster(
    array_x,
    lower_left_corner=original_raster.extent.lowerLeft,
    x_cell_size=original_raster.meanCellWidth,
    y_cell_size=original_raster.meanCellHeight,
    )
out_raster_y = arcpy.NumPyArrayToRaster(
    array_y,
    lower_left_corner=original_raster.extent.lowerLeft,
    x_cell_size=original_raster.meanCellWidth,
    y_cell_size=original_raster.meanCellHeight,
    )

In [13]:
out_raster_x.save("LON")
out_raster_y.save("LAT")

In [14]:
spatial_ref = arcpy.Describe("DEM").spatialReference

In [15]:
arcpy.DefineProjection_management("LON", spatial_ref)
arcpy.DefineProjection_management("LAT", spatial_ref)

<Result 'D:\\ArcGISProjects\\workspace\\shbyq\\feature_raster_file\\features_data_dy.gdb\\LAT'>